In [181]:
import json
import requests
import pandas as pd
import yfinance as yf

url = "https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/nyse/nyse_full_tickers.json"

resp = requests.get(url)
data = json.loads(resp.text)
pd.DataFrame(data)

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
0,A,Agilent Technologies Inc. Common Stock,$136.37,-1.12,-0.815%,940756,39963949079.00,United States,1999,Biotechnology: Laboratory Analytical Instruments,Industrials,/market-activity/stocks/a
1,AA,Alcoa Corporation Common Stock,$35.94,-0.14,-0.388%,4464704,6453350101.00,United States,2016,Aluminum,Industrials,/market-activity/stocks/aa
2,AACT,Ares Acquisition Corporation II Class A Ordina...,$10.615,0.00,0.00%,162678,0.00,,2023,Blank Checks,Finance,/market-activity/stocks/aact
3,AAN,Aarons Holdings Company Inc. Common Stock,$7.32,-0.11,-1.48%,181576,222245697.00,United States,2020,Diversified Commercial Services,Miscellaneous,/market-activity/stocks/aan
4,AAP,Advance Auto Parts Inc.,$74.71,-1.82,-2.378%,1434266,4449058348.00,United States,,Auto & Home Supply Stores,Consumer Discretionary,/market-activity/stocks/aap
...,...,...,...,...,...,...,...,...,...,...,...,...
2804,ZTR,Virtus Total Return Fund Inc.,$5.38,0.00,0.00%,471174,0.00,United States,1988,Investment Managers,Finance,/market-activity/stocks/ztr
2805,ZTS,Zoetis Inc. Class A Common Stock,$153.36,2.48,1.644%,4566786,70218500756.00,United States,2013,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/zts
2806,ZUO,Zuora Inc. Class A Common Stock,$9.86,-0.17,-1.695%,1453973,1439560000.00,United States,2018,Computer Software: Prepackaged Software,Technology,/market-activity/stocks/zuo
2807,ZVIA,Zevia PBC Class A Common Stock,$0.8193,-0.0104,-1.253%,86750,58866734.00,United States,2021,Beverages (Production/Distribution),Consumer Staples,/market-activity/stocks/zvia


In [182]:
yf.Ticker("RDY").info

{'address1': '8-2-337, Road No. 3',
 'address2': 'Banjara Hills',
 'city': 'Hyderabad',
 'zip': '500034',
 'country': 'India',
 'phone': '91 40 4900 2900',
 'fax': '91 40 4900 2999',
 'website': 'https://www.drreddys.com',
 'industry': 'Drug Manufacturers—Specialty & Generic',
 'sector': 'Healthcare',
 'longBusinessSummary': "Dr. Reddy's Laboratories Limited, together with its subsidiaries, operates as an integrated pharmaceutical company worldwide. It operates through Global Generics, Pharmaceutical Services and Active Ingredients (PSAI), and Others segments. The company's Global Generics segment manufactures and markets prescription and over-the-counter finished pharmaceutical products that are marketed under a brand name or as a generic finished dosages with therapeutic equivalence to branded formulations. This segment also engages in the biologics business. The PSAI segment manufactures and markets active pharmaceutical ingredients and intermediates, which are principal ingredients

In [152]:
import yfinance as yf

tickers = ["AES", "AESI", "AAPL", "META"]

f = []
for ticker in tickers:
    a = ["symbol", "beta", "revenueGrowth", "priceToBook", "debtToEquity", "profitMargins", "quickRatio", 
         "fiftyDayAverage", "pegRatio"]
    try:
        f.append(dict(filter(lambda item: item[0] in a, yf.Ticker(ticker).info.items())))
    except KeyError:
        continue
    except ValueError:
        continue

f = pd.DataFrame(f)
# f.insert(0,"symbol", tickers)

a = ["symbol", "beta", "revenueGrowth", "priceToBook", "debtToEquity", "profitMargins", "quickRatio", 
         "fiftyDayAverage", "pegRatio"]
f[a]

,symbol,beta,revenueGrowth,priceToBook,debtToEquity,profitMargins,quickRatio,fiftyDayAverage,pegRatio
0,AES,1.087,-0.030,6.952110,379.595,0.01966,0.347,16.4654,1.21
1,AESI,NaN,-0.058,2.669433,20.510,0.26059,3.039,21.1666,-30.00
2,AAPL,1.276,0.021,35.445440,145.803,0.26163,0.920,173.8860,2.69
3,META,NaN,0.273,7.512340,25.168,0.32063,2.546,495.4192,1.09


In [125]:
a = ["beta", "revenueGrowth", "priceToBook", "debtToEquity", "profitMargins", "quickRatio", 
    "fiftyDayAverage", "pegRatio"]

tick = {"AAPL": dict(filter(lambda item: item[0] in a, yf.Ticker("AAPL").info.items()))}

pd.DataFrame(tick)

,AAPL
beta,1.27600
debtToEquity,145.80300
fiftyDayAverage,173.88600
pegRatio,2.69000
priceToBook,35.44544
profitMargins,0.26163
quickRatio,0.92000
revenueGrowth,0.02100


In [180]:
z = [1,2,3,4]
a = [5,6,7,8]

b = dict(zip(z,a))

dict(list(b.items())[1:4])

{2: 6, 3: 7, 4: 8}

In [37]:
import subprocess
import os

directory_location = os.getcwd()

symbols = subprocess.run(['python', directory_location + "\\data_cleaning.py"], text=True, stdout=subprocess.PIPE)

In [49]:
tick = symbols.stdout.strip('""[]\n').split(', ')

list_result_without_quotes = [element.strip("'\"") for element in tick]

list_result_without_quotes 

['DOCS', 'ESNT', 'TRNO', 'ALKS', 'DLO', 'INCY', 'LOPE', 'MKTX', 'SEIC']